In [1]:
import torch

In [1]:
import numpy as np

# Parameters range
iteration_range = (5000, 120000)
num_processes_range = (1, 16)
size_range = (50, 500)

# Normalization function
def normalize_params(iteration, num_processes, size):
    normalized_iteration = (iteration - iteration_range[0]) / (iteration_range[1] - iteration_range[0])
    normalized_num_processes = (num_processes - num_processes_range[0]) / (num_processes_range[1] - num_processes_range[0])
    normalized_size = (size - size_range[0]) / (size_range[1] - size_range[0])
    return [normalized_iteration, normalized_num_processes, normalized_size]

# Generate dataset
def generate_dataset(num_samples):
    samples = []
    for _ in range(num_samples):
        iteration = np.random.randint(*iteration_range)
        num_processes = np.random.choice(*num_processes_range)
        size = np.random.choice(*size_range)
        samples.append(normalize_params(iteration, num_processes, size))
    return np.array(samples)

# def generate_dataset(num_samples):
#     samples = []
#     for _ in range(num_samples):
#         iteration = np.random.randint(*iteration_range)
#         num_processes = np.random.choice(num_processes_options)
#         size = np.random.choice(size_options)
#         samples.append([iteration, num_processes, size])
#     return np.array(samples)


# Generate dataset
num_samples = 400000  # You can adjust the number of samples
dataset = generate_dataset(num_samples)
dataset

TypeError: 'tuple' object is not callable

In [3]:
import numpy as np
np.random.randint(*iteration_range)

7359

In [4]:
from sklearn.model_selection import train_test_split

# Splitting the dataset
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

# Converting to PyTorch tensors and moving to GPU if available
train_tensor = torch.Tensor(train_data).to('cuda' if torch.cuda.is_available() else 'cpu')
test_tensor = torch.Tensor(test_data).to('cuda' if torch.cuda.is_available() else 'cpu')


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim

# Assuming inputs are already normalized to a 0-1 range for simplicity
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(3, 128), # Input layer: 3 inputs
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 128), # Hidden layer
            nn.ReLU(),
            nn.Linear(128, 3), # Output layer: Encoded form
            nn.Sigmoid() # Ensuring output is in the 0-1 range
            # nn.ReLU()
        )
        
    def forward(self, x):
        return self.network(x)

class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(3, 128), # Input layer: Encoded form
            nn.ReLU(),
            nn.Linear(128, 128), # Hidden layer
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 3), # Output layer: Predicting original inputs
            nn.Sigmoid() # Assuming the original inputs were normalized
            # nn.ReLU()
        )
        
    def forward(self, x):
        return self.network(x)


In [6]:
encoder = Encoder().to('cuda' if torch.cuda.is_available() else 'cpu')
decoder = Decoder().to('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
# Loss and Optimizer
criterion = nn.MSELoss()
encoder_optimizer = optim.Adam(encoder.parameters(), lr=0.0001)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=0.0001)

In [8]:
# Training settings
batch_size = 512
epochs = 100

# Training and validation loop
for epoch in range(epochs):
    # Training
    encoder.train()
    decoder.train()
    train_loss_encoder, train_loss_decoder = 0, 0
    for i in range(0, len(train_tensor), batch_size):
        batch = train_tensor[i:i+batch_size]

        # Train Decoder (Adversary) to predict original inputs from the encoded forms
        encoded = encoder(batch).detach()  # Detach to prevent gradients from flowing into the encoder
        decoded = decoder(encoded)
        decoder_loss = criterion(decoded, batch)

        decoder_optimizer.zero_grad()
        decoder_loss.backward()
        decoder_optimizer.step()
        train_loss_decoder += decoder_loss.item()

        # Train Encoder to obfuscate the input against the Decoder
        encoded = encoder(batch)
        decoded = decoder(encoded)
        encoder_loss = -criterion(decoded, batch)  # Using negative loss as an adversarial goal

        encoder_optimizer.zero_grad()
        encoder_loss.backward()
        encoder_optimizer.step()
        train_loss_encoder += encoder_loss.item()
    
    # Validation
    encoder.eval()
    decoder.eval()
    with torch.no_grad():
        encoded_test = encoder(test_tensor)
        decoded_test = decoder(encoded_test)
        test_loss = criterion(decoded_test, test_tensor).item()
    
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Train Encoder Loss: {train_loss_encoder / len(train_tensor)}, Train Decoder Loss: {train_loss_decoder / len(train_tensor)}, Test Loss: {test_loss}")


Epoch 0, Train Encoder Loss: -0.0002053499699337408, Train Decoder Loss: 0.00020535899156238883, Test Loss: 0.10529646277427673
Epoch 10, Train Encoder Loss: -0.00020531660446431488, Train Decoder Loss: 0.00020533443870954216, Test Loss: 0.10516113042831421
Epoch 20, Train Encoder Loss: -0.0002053308806847781, Train Decoder Loss: 0.00020533473554532975, Test Loss: 0.10526196658611298
Epoch 30, Train Encoder Loss: -0.00020533572835847736, Train Decoder Loss: 0.00020533722250256687, Test Loss: 0.10525839775800705
Epoch 40, Train Encoder Loss: -0.0002053348695859313, Train Decoder Loss: 0.0002053358337841928, Test Loss: 0.1052589863538742
Epoch 50, Train Encoder Loss: -0.0002053348797140643, Train Decoder Loss: 0.00020533597036264837, Test Loss: 0.10525906831026077
Epoch 60, Train Encoder Loss: -0.00020533458802383394, Train Decoder Loss: 0.00020533536872826516, Test Loss: 0.10525902360677719
Epoch 70, Train Encoder Loss: -0.0002053344988496974, Train Decoder Loss: 0.00020533518367446958,

In [9]:
train_tensor

tensor([[0.2740, 0.1429, 0.5000],
        [0.7104, 0.1429, 0.5000],
        [0.6314, 1.0000, 0.2500],
        ...,
        [0.0125, 1.0000, 0.2500],
        [0.8585, 0.5714, 1.0000],
        [0.4269, 0.8571, 1.0000]], device='cuda:0')

In [12]:
t = torch.tensor([0.1, 0.25, 1]).to('cuda')
t

tensor([0.1000, 0.2500, 1.0000], device='cuda:0')

In [13]:
encoder(t)

tensor([1.0000e+00, 4.5737e-10, 1.0000e+00], device='cuda:0',
       grad_fn=<SigmoidBackward0>)

In [15]:
decoder(encoder(t))

tensor([0.4990, 0.4995, 0.5003], device='cuda:0', grad_fn=<SigmoidBackward0>)